In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import csv
import optuna

In [2]:
# 셀에서 출력 정확도 설정.
%precision %.3f

'%.3f'

In [3]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

documents = documents[:77] + documents[-77:]

In [4]:
# 전문가 평점 3점 이상인 상위 175개는 label에 1, 나머지 하위는 label에 0 값 부여
# 분류 예측용 label
labels = []
for i in range(154):
    if i < 77:
        labels.append(1)
    else:
        labels.append(0)

In [5]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(documents_corpus, labels, test_size=0.3, random_state=123)

In [7]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(X_train)]

In [12]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Randomforest 하이퍼파라미터 튜닝
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)

    # Randomforest 모델 생성
    model = RandomForestClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 14:02:30,678] A new study created in memory with name: no-name-526f7f01-2a5c-4eeb-a08e-d42ff9cd2f4e
[I 2024-06-20 14:02:45,593] Trial 1 finished with value: 0.6190476190476191 and parameters: {'vector_size': 80, 'window': 15, 'min_count': 7, 'epochs': 20, 'dm': 1, 'n_estimators': 446, 'max_depth': 28, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 0.5631121513209814}. Best is trial 1 with value: 0.6190476190476191.
[I 2024-06-20 14:02:47,357] Trial 4 finished with value: 0.6 and parameters: {'vector_size': 290, 'window': 8, 'min_count': 15, 'epochs': 25, 'dm': 1, 'n_estimators': 399, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_features': 0.7890393277558926}. Best is trial 1 with value: 0.6190476190476191.
[I 2024-06-20 14:02:47,542] Trial 0 finished with value: 0.47058823529411764 and parameters: {'vector_size': 230, 'window': 11, 'min_count': 7, 'epochs': 40, 'dm': 0, 'n_estimators': 308, 'max_depth': 5, 'min_samples_split': 10, '

Best hyperparameters:  {'vector_size': 140, 'window': 3, 'min_count': 19, 'epochs': 45, 'dm': 0, 'n_estimators': 391, 'max_depth': 24, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.8727453073440441}


In [13]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# RandomForest 모델 생성
clf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                    max_depth=best_params['max_depth'],
                                    min_samples_split=best_params['min_samples_split'],
                                    min_samples_leaf=best_params['min_samples_leaf'],
                                    max_features=best_params['max_features'],
                                    random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.58      0.67        24
           1       0.66      0.83      0.73        23

    accuracy                           0.70        47
   macro avg       0.72      0.70      0.70        47
weighted avg       0.72      0.70      0.70        47



In [14]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # SVM 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-4, 1.0, log=True) if kernel in ['rbf', 'poly'] else 'scale'

    # SVM 모델 생성
    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        random_state=123
    )

    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 14:13:49,417] A new study created in memory with name: no-name-a7c87ce6-bee7-4a5e-b203-0946e6d9b4b8
[I 2024-06-20 14:13:58,111] Trial 1 finished with value: 0.6896551724137931 and parameters: {'vector_size': 180, 'window': 15, 'min_count': 11, 'epochs': 20, 'dm': 0, 'C': 3.068158185111354, 'kernel': 'rbf', 'gamma': 0.0012825877356397915}. Best is trial 1 with value: 0.6896551724137931.
[I 2024-06-20 14:14:02,393] Trial 2 finished with value: 0.6571428571428571 and parameters: {'vector_size': 300, 'window': 3, 'min_count': 4, 'epochs': 30, 'dm': 0, 'C': 0.042530308325904626, 'kernel': 'poly', 'degree': 2, 'gamma': 0.03428633477459653}. Best is trial 1 with value: 0.6896551724137931.
[I 2024-06-20 14:14:04,944] Trial 6 finished with value: 0.6571428571428571 and parameters: {'vector_size': 60, 'window': 8, 'min_count': 18, 'epochs': 45, 'dm': 0, 'C': 6.753928003135042, 'kernel': 'rbf', 'gamma': 0.00011127348037680895}. Best is trial 1 with value: 0.6896551724137931.
[I 2024

Best hyperparameters:  {'vector_size': 240, 'window': 7, 'min_count': 5, 'epochs': 25, 'dm': 1, 'C': 77.87975872156572, 'kernel': 'rbf', 'gamma': 0.09979163094399905}


In [15]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# SVM 분류 모델 생성
clf = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'], random_state=123)#degree=best_params['degree'], gamma=best_params['gamma'], 

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.79      0.79        24
           1       0.78      0.78      0.78        23

    accuracy                           0.79        47
   macro avg       0.79      0.79      0.79        47
weighted avg       0.79      0.79      0.79        47



In [17]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # XGBOOST 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    
    # XGboost 모델 생성
    model = XGBClassifier(max_depth=max_depth,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        gamma=gamma,
                        min_child_weight=min_child_weight,
                        random_state=123)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 14:25:31,487] A new study created in memory with name: no-name-92facef7-326d-48f1-adc6-0f23668e9d56
[I 2024-06-20 14:25:42,768] Trial 1 finished with value: 0.5777777777777777 and parameters: {'vector_size': 220, 'window': 11, 'min_count': 20, 'epochs': 30, 'dm': 0, 'max_depth': 13, 'n_estimators': 219, 'learning_rate': 0.034535827502878855, 'gamma': 0.8058155986775329, 'min_child_weight': 5}. Best is trial 1 with value: 0.5777777777777777.
[I 2024-06-20 14:25:45,774] Trial 4 finished with value: 0.5581395348837209 and parameters: {'vector_size': 70, 'window': 7, 'min_count': 8, 'epochs': 20, 'dm': 1, 'max_depth': 5, 'n_estimators': 246, 'learning_rate': 0.2568645114771035, 'gamma': 0.4550038339718243, 'min_child_weight': 12}. Best is trial 1 with value: 0.5777777777777777.
[I 2024-06-20 14:25:45,786] Trial 0 finished with value: 0.6571428571428571 and parameters: {'vector_size': 160, 'window': 5, 'min_count': 15, 'epochs': 45, 'dm': 0, 'max_depth': 24, 'n_estimators': 23

Best hyperparameters:  {'vector_size': 190, 'window': 5, 'min_count': 16, 'epochs': 45, 'dm': 0, 'max_depth': 6, 'n_estimators': 300, 'learning_rate': 0.1465945218861909, 'gamma': 0.12436418006609373, 'min_child_weight': 11}


In [18]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# XGBoost 모델 생성
clf = XGBClassifier(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    random_state=123
)

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.67      0.70        24
           1       0.68      0.74      0.71        23

    accuracy                           0.70        47
   macro avg       0.70      0.70      0.70        47
weighted avg       0.70      0.70      0.70        47



In [19]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # LightGBM 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 50)

    # LightGBM 모델 생성
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 14:35:40,083] A new study created in memory with name: no-name-2e4497c1-318f-4b04-bfe7-3b3bce894e59
[I 2024-06-20 14:35:55,118] Trial 6 finished with value: 0.6037735849056604 and parameters: {'vector_size': 270, 'window': 12, 'min_count': 16, 'epochs': 40, 'dm': 0, 'max_depth': 7, 'num_leaves': 73, 'learning_rate': 0.011809860873663732, 'n_estimators': 291, 'min_child_samples': 45}. Best is trial 6 with value: 0.6037735849056604.
[I 2024-06-20 14:35:56,511] Trial 1 finished with value: 0.3902439024390244 and parameters: {'vector_size': 140, 'window': 4, 'min_count': 20, 'epochs': 50, 'dm': 0, 'max_depth': 18, 'num_leaves': 81, 'learning_rate': 0.28787141717892895, 'n_estimators': 70, 'min_child_samples': 50}. Best is trial 6 with value: 0.6037735849056604.
[I 2024-06-20 14:35:56,621] Trial 5 finished with value: 0.5833333333333334 and parameters: {'vector_size': 300, 'window': 12, 'min_count': 5, 'epochs': 40, 'dm': 0, 'max_depth': 26, 'num_leaves': 194, 'learning_rate':

Best hyperparameters:  {'vector_size': 130, 'window': 4, 'min_count': 11, 'epochs': 40, 'dm': 0, 'max_depth': 24, 'num_leaves': 174, 'learning_rate': 0.27734891346802193, 'n_estimators': 426, 'min_child_samples': 37}


In [26]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# LightGBM 모델 생성
clf = LGBMClassifier(max_depth=best_params['max_depth'], num_leaves=best_params['num_leaves'], 
                    learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], 
                    min_child_samples=best_params['min_child_samples'], random_state=123)


# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

[LightGBM] [Info] Number of positive: 54, number of negative: 53
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4845
[LightGBM] [Info] Number of data points in the train set: 107, number of used features: 130
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504673 -> initscore=0.018692
[LightGBM] [Info] Start training from score 0.018692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [27]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 3, 15)
    min_count = trial.suggest_int('min_count', 1, 20)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    dm = trial.suggest_int('dm', 0, 1)

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, seed=123)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Logistic Regression 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Logistic Regression 모델 생성
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=123
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-20 14:47:18,092] A new study created in memory with name: no-name-454dadf7-f4af-4e39-ab40-b28cc886dc71
[I 2024-06-20 14:47:25,422] Trial 1 finished with value: 0.6 and parameters: {'vector_size': 140, 'window': 8, 'min_count': 12, 'epochs': 20, 'dm': 0, 'C': 3.441159238120879, 'penalty': 'l2', 'solver': 'liblinear'}. Best is trial 1 with value: 0.6.
[I 2024-06-20 14:47:28,820] Trial 3 finished with value: 0.8 and parameters: {'vector_size': 190, 'window': 6, 'min_count': 17, 'epochs': 30, 'dm': 0, 'C': 0.6976522881420413, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 3 with value: 0.8.
[I 2024-06-20 14:47:31,844] Trial 2 finished with value: 0.6274509803921569 and parameters: {'vector_size': 280, 'window': 8, 'min_count': 11, 'epochs': 35, 'dm': 0, 'C': 2.1779697826623847, 'penalty': 'l1', 'solver': 'liblinear'}. Best is trial 3 with value: 0.8.
[I 2024-06-20 14:47:33,751] Trial 4 finished with value: 0.6222222222222222 and parameters: {'vector_size': 250, 'window':

Best hyperparameters:  {'vector_size': 190, 'window': 6, 'min_count': 17, 'epochs': 30, 'dm': 0, 'C': 0.6976522881420413, 'penalty': 'l1', 'solver': 'liblinear'}


In [28]:
best_params = study.best_params

# Doc2Vec 모델 생성
model = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], seed=123)

# Logistic Regressor 모델 생성
clf = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'], random_state=123)

# Doc2Vec 벡터 생성
X_train_vec = [model.infer_vector(doc) for doc in X_train]
X_test_vec = [model.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf.fit(X_train_vec, y_train)
y_pred = clf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.67      0.73        24
           1       0.70      0.83      0.76        23

    accuracy                           0.74        47
   macro avg       0.75      0.75      0.74        47
weighted avg       0.75      0.74      0.74        47



In [24]:
# import pickle

# with open('d2v_model_1245.pkl', 'wb') as f:
#     pickle.dump(model, f)
    
# with open('svc_model_1245.pkl', 'wb') as f:
#     pickle.dump(clf, f)